1. Ładowanie i wstępna obróbka danych

In [1]:
import pandas as pd

1.1. Wczytanie danych z plików do dataframe

In [2]:
twitter_path = 'C:\\Users\\atago\\PycharmProjects\\ProjektKoncowy\\datasets\\tweets.csv'
bitcoin_path = 'C:\\Users\\atago\\PycharmProjects\\ProjektKoncowy\\datasets\\BTC-USD.csv'

Wczytanie danych z twittera (chunki danych)
Ograniczenie czasowe (dla optymalizacji ładowania, wynika z przeprowadzonych testów):
'2014-09-17' - '2019-05-27'

In [3]:
# Ograniczenie liczby wczytywanych kolumn
usecols_twitter = ['timestamp', 'text']
dtype_mapping = {
    'timestamp': str,
    'text': str
}

# Wczytaj dane z pliku, ale nie wczytuj wszystkich chunków na raz
data = pd.read_csv(twitter_path, sep=';', usecols=usecols_twitter, dtype=dtype_mapping, chunksize=100000, engine='python', parse_dates=['timestamp'])

# Inicjalizuj pustą listę, która będzie przechowywać filtrowane chunki danych
filtered_data = []

# Iteruj przez chunki danych i wybierz tylko te z odpowiednim zakresem dat
for chunk in data:
    filtered_chunk = chunk[(chunk['timestamp'] >= '2014-09-17') & (chunk['timestamp'] <= '2019-05-27')]
    filtered_data.append(filtered_chunk)

# Po zakończeniu iteracji, połącz wszystkie filtrowane chunki w jedną ramkę danych
df_t = pd.concat(filtered_data)

In [4]:
# konwersja timestamp do daty
df_t['timestamp'] = pd.to_datetime(df_t['timestamp'])
df_t['Date'] = df_t['timestamp'].dt.strftime('%Y-%m-%d')
df_t.drop(['timestamp'], axis=1, inplace=True)

In [5]:
#zamiana kolejności kolumn
column1 = 'Date'
column2 = 'text'
colnames = df_t.columns.tolist()
index1, index2 = colnames.index(column1), colnames.index(column2)
colnames[index2], colnames[index1] = colnames[index1], colnames[index2]
df_t = df_t[colnames]

In [6]:
df_t.head()

,Date,text
11,2019-05-21,"BTC IS STILL GOING STRONG!!\n\nThus, we are gi..."
12,2019-05-22,BestMixer has been seized by the Dutch Police ...
15,2019-05-10,share
20,2019-05-26,$HOT $HOT $HOT 🍀\n\n🚀🚀🚀🚀🚀\n\n🔥🔥🔥🔥🔥 https://t.c...
22,2019-05-02,https://t.co/Ip0ph8uZYn\n\n#cryptocurrency #dr...


Wczytanie danych bitcoina

In [7]:
# Wczytanie datasetu z ceną Bitcoina i zmiana nazw kolumn
usecols_bitcoin = ['Date', 'Adj Close']
df_b = pd.read_csv(bitcoin_path, sep =',', usecols=usecols_bitcoin, encoding='utf-8')
df_b.rename(columns={'Adj Close': 'bitcoin_price'}, inplace=True)

Czyszczenie duplikatów i pustych wierszy w zbiorze twitter

In [8]:
na_count = df_t['text'].isna().sum()
duplicates_count = df_t['text'].duplicated().sum()
print(f'Na values count: {na_count}')
print(f'Duplicated values count: {duplicates_count}')

Na values count: 722
Duplicated values count: 375006


In [9]:
# Czyszczenie duplikatów
duplicates = df_t['text'].duplicated()
df_t = df_t.drop_duplicates(subset=['text'])
# Czyszczenie pustych wierszy
df_t.dropna(subset=['text'], inplace=True)

Dodanie kolumny z trendem ceny bitcoina

In [10]:
def get_trend(df):
    df['bitcoin_trend'] = df['bitcoin_price'].shift(-1) - df['bitcoin_price']
    df['bitcoin_trend'] = df['bitcoin_trend'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    return df

In [11]:
#dodanie bitcoin_trend i usunięcie wierszy z wartością 0
get_trend(df_b)
df_b = df_b[df_b['bitcoin_trend'] != 0]
print(df_b['bitcoin_trend'].unique())

[-1  1]


Połączenie zbiorów danych po dacie

In [12]:
data = df_t.merge(df_b, on = 'Date', how = 'inner', left_index=False)

In [13]:
data.head()

,Date,text,bitcoin_price,bitcoin_trend
0,2019-05-21,"BTC IS STILL GOING STRONG!!\n\nThus, we are gi...",7963.327637,-1
1,2019-05-21,IDAX #Japan #エアドロップ\n\n海外取引所 #IDAX 日本語アカウントのオー...,7963.327637,-1
2,2019-05-21,Multiply your Bitcoin and Ethereum: https://t...,7963.327637,-1
3,2019-05-21,Game of DOGEs begun! Become a champion of the ...,7963.327637,-1
4,2019-05-21,"$BTC levels \n\nSupport: 7600, 7200, 6800, 640...",7963.327637,-1


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3932611 entries, 0 to 3932610
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Date           object 
 1   text           object 
 2   bitcoin_price  float64
 3   bitcoin_trend  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 120.0+ MB


1.2. Wczytanie danych po wstępnej obróbce do pliku

In [15]:
data.to_csv('data_after_preparation.csv', index=False)